# FedScope Employment Data Comparison: September 2024 vs March 2025

## ⚠️ REDACTION ISSUES IN MARCH 2025 DATA ⚠️

March 2025 has significant redaction that makes direct comparisons problematic.

This notebook demonstrates:
1. How to analyze redaction patterns
2. How to work around redaction by excluding affected agencies

## Functions for Working Around Redaction

- `get_agencies_with_redacted_data(df, field)`: Identify agencies with redacted data
- **Alternatively**: You can hard-code excluding AF, AR, DD, NV

## Important Notes

**Category names changed between time periods**, so use coded fields (`edlvl`, `toa`, `agy`) for accurate matching, then map to descriptive names from both datasets.

## Data Sources

- **March 2025**: `fedscope_employment_March_2025.parquet` 
- **September 2024**: `fedscope_employment_September_2024.parquet`

In [1]:
import pandas as pd
import numpy as np
from great_tables import GT, html, md, style, loc
from typing import Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Functions (Expand to See)

In [2]:
def load_fedscope_data() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load March 2025 and September 2024 FedScope data from GitHub.
    
    Returns:
        Tuple of (march_2025_df, september_2024_df)
    """
    # GitHub URLs for the parquet files
    base_url = "https://github.com/abigailhaddad/fedscope_employment/raw/main/fedscope_data/parquet/"
    
    march_url = f"{base_url}fedscope_employment_March_2025.parquet"
    sept_url = f"{base_url}fedscope_employment_September_2024.parquet"
    
    try:
        df_march = pd.read_parquet(march_url)
        df_sept = pd.read_parquet(sept_url)
        return df_march, df_sept
        
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None, None

def create_default_gt_format() -> dict:
    """
    Create a default configuration for great-tables formatting that can be reused.
    
    Returns:
        Dictionary with default formatting options
    """
    return {
        'table_font_size': '12px',
        'heading_title_font_size': '16px', 
        'heading_subtitle_font_size': '14px',
        'number_decimals': 0,
        'percent_decimals': 1,
        'color_palette': ['#e53e3e', '#ffffff', '#38a169'],  # red, white, green
        'show_thousands_sep': True,
        'force_percent_sign': True
    }

def clean_agency_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean agency names by removing prefixes from September 2024 data.
    September 2024 format: 'VATA-VETERANS HEALTH ADMINISTRATION'
    March 2025 format: 'VETERANS HEALTH ADMINISTRATION'
    """
    df_clean = df.copy()
    df_clean['agysubt'] = df_clean['agysubt'].astype(str).str.replace(r'^[A-Z0-9]+-', '', regex=True)
    return df_clean

def clean_education_level_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean education level names by removing numeric prefixes from September 2024 data.
    September 2024 format: '13-BACHELOR\'S DEGREE'
    March 2025 format: 'BACHELOR\'S DEGREE'
    """
    df_clean = df.copy()
    # Only remove numeric prefixes (1-2 digits followed by dash)
    df_clean['edlvlt'] = df_clean['edlvlt'].astype(str).str.replace(r'^\d{1,2}-', '', regex=True)
    # Also remove ** prefix
    df_clean['edlvlt'] = df_clean['edlvlt'].astype(str).str.replace(r'^\*\*-', '', regex=True)
    return df_clean

def clean_appointment_type_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean appointment type names by removing numeric prefixes from September 2024 data.
    September 2024 format: '10-Competitive Service - Career'
    March 2025 format: 'CAREER (COMPETITIVE SERVICE PERMANENT)'
    """
    df_clean = df.copy()
    # Only remove numeric prefixes (1-2 digits followed by dash)
    df_clean['toat'] = df_clean['toat'].astype(str).str.replace(r'^\d{1,2}-', '', regex=True)
    # Also remove ** prefix
    df_clean['toat'] = df_clean['toat'].astype(str).str.replace(r'^\*\*-', '', regex=True)
    return df_clean

def clean_occupation_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean occupation names by removing prefixes from September 2024 data.
    September 2024 format: '0610-NURSE'
    March 2025 format: 'NURSE'
    """
    df_clean = df.copy()
    df_clean['occt'] = df_clean['occt'].astype(str).str.replace(r'^[A-Z0-9]+-', '', regex=True)
    return df_clean

def clean_category_names(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Generic function to clean category names by removing prefixes.
    September 2024 data has prefixes like 'VATA-' or '0610-' that need to be removed.
    """
    df_clean = df.copy()
    
    # For education and appointment types, use more specific cleaning
    if column == 'edlvlt':
        df_clean = clean_education_level_names(df_clean)
    elif column == 'toat':
        df_clean = clean_appointment_type_names(df_clean)
    else:
        # For other fields like agencies, use the original broader cleaning
        df_clean[column] = df_clean[column].astype(str).str.replace(r'^[A-Z0-9]+-', '', regex=True)
    
    return df_clean

def create_comparison_table(df_march: pd.DataFrame, df_sept: pd.DataFrame, 
                          group_column: str, title: str, filter_func=None, 
                          custom_sort_func=None, clean_names=True, text_column=None) -> pd.DataFrame:
    """
    Create a comparison table between March 2025 and September 2024 data.
    
    Args:
        df_march: March 2025 data
        df_sept: September 2024 data
        group_column: Column to group by (code field)
        title: Title for the table
        filter_func: Optional function to filter data before grouping
        text_column: Text description column to add readable names
    
    Returns:
        DataFrame with comparison data
    """
    # Apply filter if provided
    if filter_func:
        df_march_filtered = df_march[filter_func(df_march)].copy()
        df_sept_filtered = df_sept[filter_func(df_sept)].copy()
    else:
        df_march_filtered = df_march.copy()
        df_sept_filtered = df_sept.copy()
    
    # Clean category names only if needed (agencies and occupations have prefixes)
    # September 2024 data has prefixes like 'VATA-' or '0610-' that need to be removed
    if clean_names:
        df_march_filtered = clean_category_names(df_march_filtered, group_column)
        df_sept_filtered = clean_category_names(df_sept_filtered, group_column)
    
    # Convert employment to numeric (handling string values)
    df_march_filtered['employment_num'] = pd.to_numeric(
        df_march_filtered['employment'].replace(['REDACTED', '*****', 'nan', ''], '1'), 
        errors='coerce'
    ).fillna(1)
    
    df_sept_filtered['employment_num'] = pd.to_numeric(
        df_sept_filtered['employment'].replace(['REDACTED', '*****', 'nan', ''], '1'), 
        errors='coerce'
    ).fillna(1)
    
    # Group and sum employment counts
    march_counts = df_march_filtered.groupby(group_column)['employment_num'].sum().round().astype(int)
    sept_counts = df_sept_filtered.groupby(group_column)['employment_num'].sum().round().astype(int)
    
    # Get all unique categories from both periods
    all_categories = sorted(set(march_counts.index) | set(sept_counts.index))
    
    # Create name mapping using March 2025 names first, then September 2024 as fallback
    name_mapping = {}
    if text_column:
        # Get March 2025 code-to-name mapping
        march_mapping = df_march_filtered.groupby(group_column)[text_column].first().to_dict()
        # Get September 2024 code-to-name mapping (cleaned if needed)
        if clean_names:
            df_sept_clean = clean_category_names(df_sept_filtered, text_column)
            sept_mapping = df_sept_clean.groupby(group_column)[text_column].first().to_dict()
        else:
            sept_mapping = df_sept_filtered.groupby(group_column)[text_column].first().to_dict()
        
        # Create combined mapping: use March 2025 name if available, otherwise use September 2024
        for category in all_categories:
            if category in march_mapping and march_mapping[category] not in ['REDACTED', '', 'nan']:
                name_mapping[category] = march_mapping[category]
            elif category in sept_mapping and sept_mapping[category] not in ['REDACTED', '', 'nan']:
                name_mapping[category] = sept_mapping[category]
            else:
                name_mapping[category] = str(category)  # Use code as fallback
    
    # Create comparison dataframe with proper matching
    comparison_data = []
    redacted_data = None
    
    for category in all_categories:
        march_val = march_counts.get(category, 0)
        sept_val = sept_counts.get(category, 0)
        
        # Get readable name
        category_name = name_mapping.get(category, str(category)) if text_column else str(category)
        
        # Handle REDACTED separately to put at bottom
        if category == 'REDACTED' or category_name == 'REDACTED':
            if text_column:
                redacted_data = {
                    'Code': category,
                    'Category': category_name,
                    'Sep_2024': sept_val,
                    'Mar_2025': march_val
                }
            else:
                redacted_data = {
                    'Category': category_name,
                    'Sep_2024': sept_val,
                    'Mar_2025': march_val
                }
        else:
            if text_column:
                comparison_data.append({
                    'Code': category,
                    'Category': category_name,
                    'Sep_2024': sept_val,
                    'Mar_2025': march_val
                })
            else:
                comparison_data.append({
                    'Category': category_name,
                    'Sep_2024': sept_val,
                    'Mar_2025': march_val
                })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Add REDACTED at the bottom if it exists
    if redacted_data:
        comparison_df = pd.concat([comparison_df, pd.DataFrame([redacted_data])], ignore_index=True)
    
    # Calculate changes
    comparison_df['Change'] = comparison_df['Mar_2025'] - comparison_df['Sep_2024']
    comparison_df['Pct_Change'] = np.where(
        comparison_df['Sep_2024'] > 0,
        (comparison_df['Change'] / comparison_df['Sep_2024']) * 100,
        np.where(comparison_df['Mar_2025'] > 0, 100, 0)
    )
    
    # Add totals row at the bottom
    total_sept = comparison_df['Sep_2024'].sum()
    total_march = comparison_df['Mar_2025'].sum()
    total_change = total_march - total_sept
    total_pct_change = (total_change / total_sept) * 100 if total_sept > 0 else 0
    
    # Create totals row
    if text_column:
        totals_row = {
            'Code': 'TOTAL',
            'Category': 'TOTAL',
            'Sep_2024': total_sept,
            'Mar_2025': total_march,
            'Change': total_change,
            'Pct_Change': total_pct_change
        }
    else:
        totals_row = {
            'Category': 'TOTAL',
            'Sep_2024': total_sept,
            'Mar_2025': total_march,
            'Change': total_change,
            'Pct_Change': total_pct_change
        }
    
    # Add totals row to dataframe
    comparison_df = pd.concat([comparison_df, pd.DataFrame([totals_row])], ignore_index=True)
    
    # Apply custom sorting if provided, otherwise sort by March 2025 count
    if custom_sort_func:
        comparison_df = custom_sort_func(comparison_df)
    else:
        comparison_df = comparison_df.sort_values('Mar_2025', ascending=False)
    
    return comparison_df

def format_comparison_table(df: pd.DataFrame, title: str, custom_format: dict = None) -> GT:
    """
    Format a comparison dataframe using great_tables with configurable formatting.
    
    Args:
        df: Comparison dataframe
        title: Table title
        custom_format: Optional custom formatting dict, uses default if None
    
    Returns:
        GT table object
    """
    # Use default format if none provided
    fmt = custom_format if custom_format else create_default_gt_format()
    
    # Reset index to avoid indexing issues with great_tables
    df_clean = df.reset_index(drop=True)
    
    # Calculate color domains excluding TOTAL rows (handle empty dataframes)
    non_total_df = df_clean[df_clean['Category'] != 'TOTAL']
    change_max = non_total_df['Change'].abs().max() if len(non_total_df) > 0 else 1
    pct_max = non_total_df['Pct_Change'].abs().max() if len(non_total_df) > 0 else 1
    
    # Set up column labels based on whether we have a Code column
    if 'Code' in df_clean.columns:
        col_labels = {
            'Code': "Code",
            'Category': "Category",
            'Sep_2024': "Sep 2024",
            'Mar_2025': "Mar 2025", 
            'Change': "Change",
            'Pct_Change': "% Change"
        }
        right_align_cols = ["Code", "Sep_2024", "Mar_2025", "Change", "Pct_Change"]
    else:
        col_labels = {
            'Category': "Category",
            'Sep_2024': "Sep 2024",
            'Mar_2025': "Mar 2025", 
            'Change': "Change",
            'Pct_Change': "% Change"
        }
        right_align_cols = ["Sep_2024", "Mar_2025", "Change", "Pct_Change"]
    
    gt_table = (
        GT(df_clean)
        .tab_header(
            title=html(f"<strong>{title}</strong>"),
            subtitle="September 2024 vs March 2025 Comparison"
        )
        .cols_label(**col_labels)
        .fmt_number(
            columns=["Sep_2024", "Mar_2025", "Change"],
            use_seps=fmt['show_thousands_sep'],
            decimals=fmt['number_decimals']
        )
        .fmt_number(
            columns=["Pct_Change"],
            decimals=fmt['percent_decimals'],
            force_sign=fmt['force_percent_sign']
        )
        .data_color(
            columns=["Change"],
            palette=fmt['color_palette'],
            domain=[-change_max, change_max],
            rows=lambda df: df['Category'] != 'TOTAL'
        )
        .data_color(
            columns=["Pct_Change"],
            palette=fmt['color_palette'],
            domain=[-pct_max, pct_max],
            rows=lambda df: df['Category'] != 'TOTAL'
        )
        .tab_style(
            style=style.text(weight="bold"),
            locations=loc.column_labels()
        )
        .tab_style(
            style=style.text(align="right"),
            locations=loc.body(columns=right_align_cols)
        )
        .tab_style(
            style=style.text(weight="bold"),
            locations=loc.body(rows=lambda df: df['Category'] == 'TOTAL')
        )
        .tab_options(
            table_font_size=fmt['table_font_size'],
            heading_title_font_size=fmt['heading_title_font_size'],
            heading_subtitle_font_size=fmt['heading_subtitle_font_size']
        )
    )
    
    return gt_table

def create_redaction_overview_chart(df_march: pd.DataFrame) -> GT:
    """
    Create a chart showing redaction percentages by variable for March 2025.
    Only show fields that have 1+ redactions.
    """
    # Get all text columns (ending in 't') plus key others
    all_columns = df_march.columns.tolist()
    
    # Variables to check for redaction - include ALL relevant fields
    redaction_fields = {}
    
    # Add all text columns (descriptive fields ending in 't')
    for col in all_columns:
        if col.endswith('t') and col != 'quarter':  # Exclude 'quarter' which ends in 't'
            # Create display name by removing 't' and making it readable
            display_name = col[:-1].replace('_', ' ').title()
            if col == 'agelvlt':
                display_name = 'Age Level'
            elif col == 'edlvlt':
                display_name = 'Education Level'
            elif col == 'toat':
                display_name = 'Type of Appointment'
            elif col == 'agysubt':
                display_name = 'Agency Sub-Agency'
            elif col == 'loct':
                display_name = 'Location'
            elif col == 'occt':
                display_name = 'Occupation'
            elif col == 'occfamt':
                display_name = 'Occupation Family'
            elif col == 'sallvlt':
                display_name = 'Salary Level'
            elif col == 'supervist':
                display_name = 'Supervisory Status'
            elif col == 'wrkscht':
                display_name = 'Work Schedule'
            elif col == 'wrkstatt':
                display_name = 'Work Status'
            elif col == 'stemoct':
                display_name = 'STEM Occupation'
            elif col == 'patcot':
                display_name = 'PATCO Category'
                
            redaction_fields[col] = display_name
    
    # Add other important fields
    other_fields = ['salary', 'employment', 'los']
    for field in other_fields:
        if field in all_columns:
            redaction_fields[field] = field.title()
    
    # Calculate redaction percentages and filter to only those with redactions
    redaction_data = []
    total_records = len(df_march)
    
    for field, display_name in redaction_fields.items():
        if field in df_march.columns:
            redacted_count = (df_march[field] == 'REDACTED').sum()
            
            # Only include fields with 1+ redactions
            if redacted_count > 0:
                redaction_pct = (redacted_count / total_records) * 100
                
                redaction_data.append({
                    'Variable': display_name,
                    'Redacted_Count': redacted_count,
                    'Total_Records': total_records,
                    'Redaction_Percentage': redaction_pct
                })
    
    # Create dataframe and sort by redaction percentage
    redaction_df = pd.DataFrame(redaction_data)
    redaction_df = redaction_df.sort_values('Redaction_Percentage', ascending=False)
    
    # Format as great_tables
    redaction_table = (
        GT(redaction_df)
        .tab_header(
            title=html("<strong>March 2025 Data Redaction Overview</strong>"),
            subtitle="Fields with REDACTED values (1+ redactions only)"
        )
        .cols_label(
            Variable="Variable",
            Redacted_Count="Redacted Records",
            Total_Records="Total Records",
            Redaction_Percentage="% Redacted"
        )
        .fmt_number(
            columns=["Redacted_Count", "Total_Records"],
            use_seps=True,
            decimals=0
        )
        .fmt_number(
            columns=["Redaction_Percentage"],
            decimals=1
        )
        .tab_style(
            style=style.text(weight="bold"),
            locations=loc.column_labels()
        )
        .tab_style(
            style=style.text(align="right"),
            locations=loc.body(columns=["Redacted_Count", "Total_Records", "Redaction_Percentage"])
        )
        .tab_options(
            table_font_size="12px",
            heading_title_font_size="16px",
            heading_subtitle_font_size="14px"
        )
    )
    
    return redaction_table

def create_sample_records_table(df_march: pd.DataFrame) -> GT:
    """
    Create a table showing 5 random sample records from redacted agencies with ALL fields.
    """
    # Get redacted agencies
    redacted_agencies = get_agencies_with_redacted_data(df_march, 'agelvlt')
    redacted_data = df_march[df_march['agy'].isin(redacted_agencies)]
    
    # Sample 5 random records
    sample_records = redacted_data.sample(n=min(5, len(redacted_data)), random_state=42)
    
    # Display ALL columns - no filtering
    sample_display = sample_records.copy()
    
    # Create the table with ALL columns
    sample_table = (
        GT(sample_display)
        .tab_header(
            title=html("<strong>Sample Records from Redacted Agencies</strong>"),
            subtitle=f"5 random records from agencies with redacted data ({', '.join(redacted_agencies)}) - ALL FIELDS"
        )
        .tab_style(
            style=style.text(weight="bold"),
            locations=loc.column_labels()
        )
        .tab_style(
            style=style.fill(color="#f8f9fa"),  # Light background for entire row if it contains REDACTED
            locations=loc.body(
                rows=lambda df: df.apply(lambda row: any(str(val) == 'REDACTED' for val in row), axis=1)
            )
        )
        .tab_options(
            table_font_size="9px",  # Smaller font to fit all columns
            heading_title_font_size="14px",
            heading_subtitle_font_size="12px",
            column_labels_font_size="10px"
        )
    )
    
    return sample_table

def create_age_groups_table(df_march: pd.DataFrame, df_sept: pd.DataFrame) -> GT:
    """
    Create age groups comparison table sorted by age (20-24 first, REDACTED last).
    """
    def sort_age_groups(df):
        """Custom sort function for age groups"""
        # Define age group order 
        age_order = ['Less than 20', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', 
                    '50-54', '55-59', '60-64', '65 or more', 'Unspecified', 'REDACTED']
        
        # Create a mapping for sorting
        order_map = {age: i for i, age in enumerate(age_order)}
        
        # Add sort key
        df['sort_key'] = df['Category'].map(lambda x: order_map.get(x, 999))
        
        # Sort and remove sort key
        df_sorted = df.sort_values('sort_key').drop('sort_key', axis=1)
        return df_sorted
    
    # Create comparison data with custom sorting (don't clean age group names, no text column needed)
    age_comparison = create_comparison_table(
        df_march, df_sept, 
        group_column='agelvlt',
        title="Age Groups Comparison",
        custom_sort_func=sort_age_groups,
        clean_names=False,
        text_column=None
    )
    
    # Format as great_tables (show all age groups)
    age_table = format_comparison_table(
        age_comparison,
        "Federal Employment by Age Groups"
    )
    
    return age_table

def create_education_level_table(df_march: pd.DataFrame, df_sept: pd.DataFrame) -> GT:
    """
    Create education level comparison table sorted from least to most education.
    """
    def sort_education_levels(df):
        """Custom sort function for education levels (least to most education)"""
        # Define education order from least to most education
        education_order = [
            'NO FORMAL EDUCATION OR SOME ELEMENTARY SCHOOL - DID NOT COMPLETE',
            'ELEMENTARY SCHOOL COMPLETED - NO HIGH SCHOOL',
            'SOME HIGH SCHOOL - DID NOT COMPLETE',
            'HIGH SCHOOL GRADUATE OR CERTIFICATE OF EQUIVALENCY',
            'TERMINAL OCCUPATIONAL PROGRAM - DID NOT COMPLETE',
            'TERMINAL OCCUPATIONAL PROGRAM - CERTIFICATE OF COMPLETION, DIPLOMA OR EQUIVALENT',
            'SOME COLLEGE - LESS THAN ONE YEAR',
            'ONE YEAR COLLEGE',
            'TWO YEARS COLLEGE',
            'ASSOCIATE DEGREE',
            'THREE YEARS COLLEGE',
            'FOUR YEARS COLLEGE',
            'BACHELOR\'S DEGREE',
            'POST-BACHELOR\'S',
            'MASTER\'S DEGREE',
            'POST-MASTER\'S',
            'SIXTH-YEAR DEGREE',
            'POST-SIXTH YEAR',
            'FIRST PROFESSIONAL',
            'POST-FIRST PROFESSIONAL',
            'DOCTORATE DEGREE',
            'POST-DOCTORATE',
            'Invalid',
            'No Data Reported',
            'REDACTED'
        ]
        
        # Create a mapping for sorting
        order_map = {edu: i for i, edu in enumerate(education_order)}
        
        # Separate TOTAL row
        total_mask = df['Category'] == 'TOTAL'
        total_row = df[total_mask]
        non_total_df = df[~total_mask]
        
        # Add sort key to non-total rows
        non_total_df['sort_key'] = non_total_df['Category'].map(lambda x: order_map.get(x, 999))
        
        # Sort and remove sort key
        non_total_sorted = non_total_df.sort_values('sort_key').drop('sort_key', axis=1)
        
        # Concatenate with TOTAL at bottom
        if len(total_row) > 0:
            return pd.concat([non_total_sorted, total_row], ignore_index=True)
        else:
            return non_total_sorted
    
    # Create comparison data using education level codes (no cleaning needed)
    education_comparison = create_comparison_table(
        df_march, df_sept,
        group_column='edlvl',
        title="Education Level Comparison",
        custom_sort_func=sort_education_levels,
        clean_names=True,
        text_column='edlvlt'
    )
    
    # Format as great_tables (show all education levels)
    education_table = format_comparison_table(
        education_comparison,
        "Federal Employment by Education Level"
    )
    
    return education_table

def create_appointment_type_table(df_march: pd.DataFrame, df_sept: pd.DataFrame) -> GT:
    """
    Create type of appointment comparison table with REDACTED at bottom.
    """
    def sort_appointments(df):
        """Custom sort function to put REDACTED and TOTAL at bottom"""
        # Separate TOTAL and REDACTED from other appointment types
        total_mask = df['Category'] == 'TOTAL'
        redacted_mask = df['Category'] == 'REDACTED'
        special_mask = total_mask | redacted_mask
        
        total_rows = df[total_mask]
        redacted_rows = df[redacted_mask]
        regular_rows = df[~special_mask]
        
        # Sort regular rows by March 2025 count (descending)
        regular_sorted = regular_rows.sort_values('Mar_2025', ascending=False)
        
        # Concatenate: regular rows, then REDACTED, then TOTAL at bottom
        result_parts = [regular_sorted]
        if len(redacted_rows) > 0:
            result_parts.append(redacted_rows)
        if len(total_rows) > 0:
            result_parts.append(total_rows)
            
        return pd.concat(result_parts, ignore_index=True)
    
    # Create comparison data using appointment type codes (no cleaning needed)
    appointment_comparison = create_comparison_table(
        df_march, df_sept,
        group_column='toa',
        title="Type of Appointment Comparison",
        custom_sort_func=sort_appointments,
        clean_names=True,
        text_column='toat'
    )
    
    # Format as great_tables (show all appointment types)
    appointment_table = format_comparison_table(
        appointment_comparison,
        "Federal Employment by Type of Appointment"
    )
    
    return appointment_table

def create_location_comparison_table(df_march: pd.DataFrame, df_sept: pd.DataFrame) -> GT:
    """
    Create location comparison table showing ALL locations by employment count.
    Uses location codes for linking with readable names, excludes redacted agencies.
    """
    # Get redacted agencies dynamically
    redacted_agencies = get_agencies_with_redacted_data(df_march, 'agelvlt')
    
    # Filter out redacted agencies
    df_march_clean = df_march[~df_march['agy'].isin(redacted_agencies)].copy()
    df_sept_clean = df_sept[~df_sept['agy'].isin(redacted_agencies)].copy()
    
    def sort_locations(df):
        """Custom sort function to put REDACTED and TOTAL at bottom"""
        # Separate TOTAL and REDACTED from other locations
        total_mask = df['Category'] == 'TOTAL'
        redacted_mask = df['Category'] == 'REDACTED'
        special_mask = total_mask | redacted_mask
        
        total_rows = df[total_mask]
        redacted_rows = df[redacted_mask]
        regular_rows = df[~special_mask]
        
        # Sort regular rows by March 2025 count (descending)
        regular_sorted = regular_rows.sort_values('Mar_2025', ascending=False)
        
        # Concatenate: regular rows, then REDACTED, then TOTAL at bottom
        result_parts = [regular_sorted]
        if len(redacted_rows) > 0:
            result_parts.append(redacted_rows)
        if len(total_rows) > 0:
            result_parts.append(total_rows)
            
        return pd.concat(result_parts, ignore_index=True)
    
    # Create comparison data using location codes with text descriptions
    location_comparison = create_comparison_table(
        df_march_clean, df_sept_clean,
        group_column='loc',  # Use location code for linking
        title="Location Comparison",
        custom_sort_func=sort_locations,
        clean_names=False,
        text_column='loct'  # Use location text for readable names
    )
    
    # Format as great_tables (show ALL locations, not just top 15)
    location_table = format_comparison_table(
        location_comparison,
        "Federal Employment by Location (All Locations, Excluding Redacted Agencies)"
    )
    
    # Add footnote about excluded agencies
    excluded_agencies_text = f"Excluded agencies with redacted data: {', '.join(redacted_agencies)}"
    location_table = location_table.tab_source_note(
        source_note=excluded_agencies_text
    )
    
    return location_table

def create_salary_comparison_table(df_march: pd.DataFrame, df_sept: pd.DataFrame) -> GT:
    """
    Create salary data availability comparison table (proportion redacted/missing), excludes redacted agencies.
    """
    # Get redacted agencies dynamically
    redacted_agencies = get_agencies_with_redacted_data(df_march, 'agelvlt')
    
    # Filter out redacted agencies
    df_march_clean = df_march[~df_march['agy'].isin(redacted_agencies)].copy()
    df_sept_clean = df_sept[~df_sept['agy'].isin(redacted_agencies)].copy()
    
    # Calculate salary data availability for both periods
    salary_stats = []
    
    # March 2025 analysis
    march_total = len(df_march_clean)
    march_redacted = (df_march_clean['salary'] == 'REDACTED').sum()
    march_missing = df_march_clean['salary'].isin(['', 'nan', '*****']).sum() + df_march_clean['salary'].isna().sum()
    march_available = march_total - march_redacted - march_missing
    
    # September 2024 analysis  
    sept_total = len(df_sept_clean)
    sept_redacted = (df_sept_clean['salary'] == 'REDACTED').sum()
    sept_missing = df_sept_clean['salary'].isin(['', 'nan', '*****']).sum() + df_sept_clean['salary'].isna().sum()
    sept_available = sept_total - sept_redacted - sept_missing
    
    # Create rows for the table (without the percentage row)
    stats_data = [
        {
            'Category': 'Total Records',
            'Sep_2024': sept_total,
            'Mar_2025': march_total,
            'Change': march_total - sept_total,
            'Pct_Change': ((march_total - sept_total) / sept_total * 100) if sept_total > 0 else 0
        },
        {
            'Category': 'Salary Available',
            'Sep_2024': sept_available,
            'Mar_2025': march_available,
            'Change': march_available - sept_available,
            'Pct_Change': ((march_available - sept_available) / sept_available * 100) if sept_available > 0 else 0
        },
        {
            'Category': 'Salary Redacted',
            'Sep_2024': sept_redacted,
            'Mar_2025': march_redacted,
            'Change': march_redacted - sept_redacted,
            'Pct_Change': ((march_redacted - sept_redacted) / sept_redacted * 100) if sept_redacted > 0 else 100
        },
        {
            'Category': 'Salary Missing/Blank',
            'Sep_2024': sept_missing,
            'Mar_2025': march_missing,
            'Change': march_missing - sept_missing,
            'Pct_Change': ((march_missing - sept_missing) / sept_missing * 100) if sept_missing > 0 else 0
        }
    ]
    
    salary_df = pd.DataFrame(stats_data)
    
    # Format as great_tables
    salary_table = (
        GT(salary_df)
        .tab_header(
            title=html("<strong>Salary Data Availability Comparison</strong>"),
            subtitle="September 2024 vs March 2025 (Excluding Redacted Agencies)"
        )
        .cols_label(
            Category="Category",
            Sep_2024="Sep 2024",
            Mar_2025="Mar 2025", 
            Change="Change",
            Pct_Change="% Change"
        )
        .fmt_number(
            columns=["Sep_2024", "Mar_2025", "Change"],
            use_seps=True,
            decimals=0
        )
        .fmt_number(
            columns=["Pct_Change"],
            decimals=1,
            force_sign=True
        )
        .tab_style(
            style=style.text(weight="bold"),
            locations=loc.column_labels()
        )
        .tab_style(
            style=style.text(align="right"),
            locations=loc.body(columns=["Sep_2024", "Mar_2025", "Change", "Pct_Change"])
        )
        .tab_options(
            table_font_size="12px",
            heading_title_font_size="16px",
            heading_subtitle_font_size="14px"
        )
    )
    
    # Add footnote about excluded agencies
    excluded_agencies_text = f"Excluded agencies with redacted data: {', '.join(redacted_agencies)}"
    salary_table = salary_table.tab_source_note(
        source_note=excluded_agencies_text
    )
    
    return salary_table

def get_agencies_with_redacted_data(df: pd.DataFrame, field: str) -> list:
    """
    Identify agencies that have redacted data for a specific field.
    
    Args:
        df: DataFrame to analyze (typically March 2025 data)
        field: Field name to check for redaction (e.g., 'agelvlt', 'edlvlt', 'loct')
    
    Returns:
        List of agency codes that have redacted data in the specified field
    """
    redacted_agencies = df[df[field] == 'REDACTED']['agy'].unique()
    return sorted(redacted_agencies.tolist())

def filter_out_redacted_agencies(df_march: pd.DataFrame, df_sept: pd.DataFrame, 
                                redacted_agencies: list) -> tuple:
    """
    Filter out agencies with redacted data from both datasets.
    
    Args:
        df_march: March 2025 data
        df_sept: September 2024 data  
        redacted_agencies: List of agency codes to exclude
    
    Returns:
        Tuple of (filtered_march_df, filtered_sept_df)
    """
    df_march_clean = df_march[~df_march['agy'].isin(redacted_agencies)].copy()
    df_sept_clean = df_sept[~df_sept['agy'].isin(redacted_agencies)].copy()
    
    return df_march_clean, df_sept_clean

def create_age_groups_table_excluding_redacted_agencies(df_march: pd.DataFrame, df_sept: pd.DataFrame) -> tuple:
    """
    Example: Create age groups comparison table excluding agencies that have redacted age data.
    This demonstrates how to work around redaction issues.
    """
    # Step 1: Identify agencies with redacted age data in March 2025
    redacted_agencies = get_agencies_with_redacted_data(df_march, 'agelvlt')
    
    # Step 2: Filter out these agencies from both datasets
    df_march_clean, df_sept_clean = filter_out_redacted_agencies(df_march, df_sept, redacted_agencies)
    
    # Step 3: Create comparison table with cleaned data
    def sort_age_groups(df):
        """Custom sort function for age groups"""
        # Separate TOTAL row
        total_mask = df['Category'] == 'TOTAL'
        total_row = df[total_mask]
        non_total_df = df[~total_mask]
        
        age_order = ['Less than 20', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', 
                    '50-54', '55-59', '60-64', '65 or more', 'Unspecified', 'REDACTED']
        order_map = {age: i for i, age in enumerate(age_order)}
        non_total_df['sort_key'] = non_total_df['Category'].map(lambda x: order_map.get(x, 999))
        non_total_sorted = non_total_df.sort_values('sort_key').drop('sort_key', axis=1)
        
        # Concatenate with TOTAL at bottom
        if len(total_row) > 0:
            return pd.concat([non_total_sorted, total_row], ignore_index=True)
        else:
            return non_total_sorted
    
    # Create comparison data using cleaned datasets
    age_comparison_clean = create_comparison_table(
        df_march_clean, df_sept_clean, 
        group_column='agelvlt',
        title="Age Groups Comparison (Excluding Redacted Agencies)",
        custom_sort_func=sort_age_groups,
        clean_names=False,
        text_column=None
    )
    
    # Format as great_tables with footnote about excluded agencies
    age_table_clean = format_comparison_table(
        age_comparison_clean,
        "Federal Employment by Age Groups (Excluding Redacted Agencies)"
    )
    
    # Add footnote about excluded agencies with full names
    # Get agency code to name mapping from September 2024 data (less redacted)
    agency_mapping = df_sept.groupby('agy')['agysubt'].first().to_dict()
    excluded_agency_names = []
    for code in redacted_agencies:
        sub_agency_name = agency_mapping.get(code, code)
        
        # Extract main agency name from sub-agency name
        if isinstance(sub_agency_name, str):
            # Common patterns: "AIR FORCE ...", "U.S. ARMY ...", "NAVAL ...", etc.
            if sub_agency_name.startswith('AF') and '-' in sub_agency_name:
                # Handle format like "AF02-AIR FORCE INSPECTION AGENCY"
                main_name = sub_agency_name.split('-', 1)[1].split(' ')[0:2]  # Get "AIR FORCE"
                main_name = ' '.join(main_name)
            elif 'AIR FORCE' in sub_agency_name:
                main_name = 'AIR FORCE'
            elif 'ARMY' in sub_agency_name:
                main_name = 'ARMY'
            elif 'NAVAL' in sub_agency_name or 'NAVY' in sub_agency_name:
                main_name = 'NAVY'
            elif code == 'DD':
                main_name = 'DEPARTMENT OF DEFENSE'
            else:
                main_name = sub_agency_name
        else:
            main_name = code
            
        excluded_agency_names.append(f"{code} ({main_name})")
    
    excluded_agencies_text = f"Excluded agencies: {', '.join(excluded_agency_names)}"
    age_table_clean = age_table_clean.tab_source_note(
        source_note=excluded_agencies_text
    )
    
    return age_table_clean, redacted_agencies

# Analysis

In [3]:
# Load the data
df_march, df_sept = load_fedscope_data()

if df_march is None or df_sept is None:
    print("❌ Failed to load data. Please check your internet connection.")
else:
    print("✅ Data loaded successfully!")
    print(f"March 2025: {len(df_march):,} employees")
    print(f"September 2024: {len(df_sept):,} employees")

✅ Data loaded successfully!
March 2025: 2,289,472 employees
September 2024: 2,313,216 employees


## Redaction Overview

In [4]:
redaction_chart = create_redaction_overview_chart(df_march)
redaction_chart.show()

March 2025 Data Redaction Overview 
 
 
 Fields with REDACTED values (1+ redactions only) 
 
 
 Variable 
 Redacted Records 
 Total Records 
 % Redacted 
 
 
 
 
 Location 
 1,069,765 
 2,289,472 
 46.7 
 
 
 Salary 
 1,052,774 
 2,289,472 
 46.0 
 
 
 Age Level 
 771,384 
 2,289,472 
 33.7 
 
 
 Education Level 
 771,384 
 2,289,472 
 33.7 
 
 
 Occupation Family 
 771,384 
 2,289,472 
 33.7 
 
 
 Occupation 
 771,384 
 2,289,472 
 33.7 
 
 
 Payplan 
 771,384 
 2,289,472 
 33.7 
 
 
 Supervisory Status 
 771,384 
 2,289,472 
 33.7 
 
 
 Type of Appointment 
 771,384 
 2,289,472 
 33.7 
 
 
 Work Schedule 
 771,384 
 2,289,472 
 33.7 
 
 
 Los 
 771,384 
 2,289,472 
 33.7

## Sample Records from Redacted Agencies

In [5]:
sample_table = create_sample_records_table(df_march)
sample_table.show()

Sample Records from Redacted Agencies 
 
 
 5 random records from agencies with redacted data (AF, AR, DD, NV) - ALL FIELDS 
 
 
 dataset_key 
 quarter 
 year 
 agysub 
 loc 
 agelvl 
 edlvl 
 gsegrd 
 loslvl 
 occ 
 patco 
 pp 
 ppgrd 
 sallvl 
 stemocc 
 supervis 
 toa 
 worksch 
 workstat 
 datecode 
 employment 
 salary 
 los 
 agelvlt 
 edlvlt 
 loslvlt 
 occtyp 
 occtypt 
 occfam 
 occfamt 
 occt 
 patcot 
 pptyp 
 ppgrdt 
 ppgroup 
 ppgroupt 
 payplan 
 payplant 
 sallvlt 
 stemocct 
 supervist 
 toat 
 wstyp 
 wstypt 
 wrkscht 
 wkstatt 
 agy 
 agysubt 
 loct 
 ppt 
 wrksch 
 wkstat 
 
 
 
 
 March_2025 
 March 
 2025 
 NV24 
 REDACTED 
 
 REDACTED 
 
 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 202503 
 1 
 REDACTED 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 
 NV 
 NAVAL SEA SYSTEMS COMMAND 
 REDACTED 
 
 
 
 
 
 March_2025 
 March 
 2025 
 DD83 
 REDACTED 
 
 REDACTED 
 
 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 202503 
 1 
 REDACTED 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 
 DD 
 MILITARY TREATMENT FACILITIES UNDER DHA 
 REDACTED 
 
 
 
 
 
 March_2025 
 March 
 2025 
 ARTC 
 REDACTED 
 
 REDACTED 
 
 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 202503 
 1 
 REDACTED 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 
 AR 
 U.S. ARMY TRAINING AND DOCTRINE COMMAND 
 REDACTED 
 
 
 
 
 
 March_2025 
 March 
 2025 
 NV25 
 REDACTED 
 
 REDACTED 
 
 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 202503 
 1 
 REDACTED 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 
 NV 
 NAVAL FACILITIES ENGINEERING COMMAND 
 REDACTED 
 
 
 
 
 
 March_2025 
 March 
 2025 
 AF0D 
 REDACTED 
 
 REDACTED 
 
 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 202503 
 1 
 REDACTED 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 REDACTED 
 REDACTED 
 REDACTED 
 
 
 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 REDACTED 
 
 
 REDACTED 
 
 AF 
 U.S. AIR FORCES, EUROPE 
 REDACTED

## Age Groups Comparison

In [6]:
age_table = create_age_groups_table(df_march, df_sept)
age_table.show()

Federal Employment by Age Groups 
 
 
 September 2024 vs March 2025 Comparison 
 
 
 Category 
 Sep 2024 
 Mar 2025 
 Change 
 % Change 
 
 
 
 
 Less than 20 
 3,113 
 838 
 −2,275 
 −73.1 
 
 
 20-24 
 59,502 
 30,120 
 −29,382 
 −49.4 
 
 
 25-29 
 145,735 
 88,229 
 −57,506 
 −39.5 
 
 
 30-34 
 214,400 
 140,438 
 −73,962 
 −34.5 
 
 
 35-39 
 286,114 
 193,276 
 −92,838 
 −32.4 
 
 
 40-44 
 337,545 
 229,427 
 −108,118 
 −32.0 
 
 
 45-49 
 309,515 
 214,695 
 −94,820 
 −30.6 
 
 
 50-54 
 307,155 
 204,025 
 −103,130 
 −33.6 
 
 
 55-59 
 297,120 
 189,532 
 −107,588 
 −36.2 
 
 
 60-64 
 221,289 
 137,711 
 −83,578 
 −37.8 
 
 
 65 or more 
 131,493 
 89,797 
 −41,696 
 −31.7 
 
 
 Unspecified 
 235 
 0 
 −235 
 −100.0 
 
 
 REDACTED 
 0 
 771,384 
 771,384 
 +100.0 
 
 
 TOTAL 
 2,313,216 
 2,289,472 
 −23,744 
 −1.0

## Education Level Comparison

In [7]:
education_table = create_education_level_table(df_march, df_sept)
education_table.show()

Federal Employment by Education Level 
 
 
 September 2024 vs March 2025 Comparison 
 
 
 Code 
 Category 
 Sep 2024 
 Mar 2025 
 Change 
 % Change 
 
 
 
 
 01 
 NO FORMAL EDUCATION OR SOME ELEMENTARY SCHOOL - DID NOT COMPLETE 
 30,488 
 29,203 
 −1,285 
 −4.2 
 
 
 02 
 ELEMENTARY SCHOOL COMPLETED - NO HIGH SCHOOL 
 391 
 308 
 −83 
 −21.2 
 
 
 03 
 SOME HIGH SCHOOL - DID NOT COMPLETE 
 8,610 
 7,162 
 −1,448 
 −16.8 
 
 
 04 
 HIGH SCHOOL GRADUATE OR CERTIFICATE OF EQUIVALENCY 
 629,600 
 382,476 
 −247,124 
 −39.3 
 
 
 05 
 TERMINAL OCCUPATIONAL PROGRAM - DID NOT COMPLETE 
 8,199 
 7,151 
 −1,048 
 −12.8 
 
 
 06 
 TERMINAL OCCUPATIONAL PROGRAM - CERTIFICATE OF COMPLETION, DIPLOMA OR EQUIVALENT 
 32,627 
 23,853 
 −8,774 
 −26.9 
 
 
 07 
 SOME COLLEGE - LESS THAN ONE YEAR 
 70,308 
 51,023 
 −19,285 
 −27.4 
 
 
 08 
 ONE YEAR COLLEGE 
 50,829 
 35,373 
 −15,456 
 −30.4 
 
 
 09 
 TWO YEARS COLLEGE 
 50,917 
 34,125 
 −16,792 
 −33.0 
 
 
 10 
 ASSOCIATE DEGREE 
 143,014 
 92,924 
 −50,090 
 −35.0 
 
 
 11 
 THREE YEARS COLLEGE 
 23,490 
 16,794 
 −6,696 
 −28.5 
 
 
 12 
 FOUR YEARS COLLEGE 
 24,306 
 15,600 
 −8,706 
 −35.8 
 
 
 13 
 BACHELOR'S DEGREE 
 613,391 
 394,681 
 −218,710 
 −35.7 
 
 
 14 
 POST-BACHELOR'S 
 24,044 
 15,713 
 −8,331 
 −34.6 
 
 
 17 
 MASTER'S DEGREE 
 397,836 
 242,571 
 −155,265 
 −39.0 
 
 
 18 
 POST-MASTER'S 
 11,403 
 5,151 
 −6,252 
 −54.8 
 
 
 19 
 SIXTH-YEAR DEGREE 
 1,293 
 765 
 −528 
 −40.8 
 
 
 20 
 POST-SIXTH YEAR 
 695 
 525 
 −170 
 −24.5 
 
 
 15 
 FIRST PROFESSIONAL 
 87,090 
 81,625 
 −5,465 
 −6.3 
 
 
 16 
 POST-FIRST PROFESSIONAL 
 2,652 
 2,236 
 −416 
 −15.7 
 
 
 21 
 DOCTORATE DEGREE 
 88,848 
 69,206 
 −19,642 
 −22.1 
 
 
 22 
 POST-DOCTORATE 
 8,526 
 7,212 
 −1,314 
 −15.4 
 
 
 * 
 Invalid 
 0 
 1,823 
 1,823 
 +100.0 
 
 
 None 
 No Data Reported 
 0 
 588 
 588 
 +100.0 
 
 
 REDACTED 
 REDACTED 
 0 
 771,384 
 771,384 
 +100.0 
 
 
 ** 
 UNSPECIFIED 
 1,793 
 0 
 −1,793 
 −100.0 
 
 
 nan 
 NO EDUCATION LEVEL REPORTED 
 2,866 
 0 
 −2,866 
 −100.0 
 
 
 TOTAL 
 TOTAL 
 2,313,216 
 2,289,472 
 −23,744 
 −1.0

## Type of Appointment Comparison

In [8]:
appointment_table = create_appointment_type_table(df_march, df_sept)
appointment_table.show()

Federal Employment by Type of Appointment 
 
 
 September 2024 vs March 2025 Comparison 
 
 
 Code 
 Category 
 Sep 2024 
 Mar 2025 
 Change 
 % Change 
 
 
 
 
 10 
 CAREER (COMPETITIVE SERVICE PERMANENT) 
 1,207,127 
 717,525 
 −489,602 
 −40.6 
 
 
 38 
 OTHER (EXCEPTED SERVICE PERMANENT) 
 546,611 
 467,572 
 −79,039 
 −14.5 
 
 
 15 
 CAREER-CONDITIONAL (COMPETITIVE SERVICE PERMANENT) 
 316,334 
 173,849 
 −142,485 
 −45.0 
 
 
 48 
 OTHER (EXCEPTED SERVICE NONPERMANENT) 
 69,881 
 55,220 
 −14,661 
 −21.0 
 
 
 30 
 SCHEDULE A (EXCEPTED SERVICE PERMANENT) 
 73,711 
 51,353 
 −22,358 
 −30.3 
 
 
 40 
 SCHEDULE A (EXCEPTED SERVICE NONPERMANENT) 
 28,094 
 15,843 
 −12,251 
 −43.6 
 
 
 20 
 NONPERMANENT (COMPETITTIVE SERVICE NONPERMANENT) 
 42,872 
 14,945 
 −27,927 
 −65.1 
 
 
 50 
 CAREER (SENIOR EXECUTIVE SERVICE PERMANENT) 
 7,951 
 6,720 
 −1,231 
 −15.5 
 
 
 35 
 SCHEDULE D (EXCEPTED SERVICE PERMANENT) 
 6,939 
 5,328 
 −1,611 
 −23.2 
 
 
 32 
 SCHEDULE B (EXCEPTED SERVICE PERMANENT) 
 5,022 
 4,100 
 −922 
 −18.4 
 
 
 42 
 SCHEDULE B (EXCEPTED SERVICE NONPERMANENT) 
 2,733 
 2,514 
 −219 
 −8.0 
 
 
 45 
 SCHEDULE D (EXCEPTED SERVICE NONPERMANENT) 
 1,864 
 881 
 −983 
 −52.7 
 
 
 None 
 Invalid 
 0 
 802 
 802 
 +100.0 
 
 
 44 
 SCHEDULE C (EXCEPTED SERVICE NONPERMANENT) 
 1,655 
 690 
 −965 
 −58.3 
 
 
 55 
 NONCAREER (SENIOR EXECUTIVE SERVICE PERMANENT) 
 765 
 346 
 −419 
 −54.8 
 
 
 46 
 EXECUTIVE (EXCEPTED SERVICE NONPERMANENT) 
 705 
 326 
 −379 
 −53.8 
 
 
 60 
 LIMITED TERM (SENIOR EXECUTIVE SERVICE NONPERMANENT) 
 64 
 48 
 −16 
 −25.0 
 
 
 36 
 EXECUTIVE (EXCEPTED SERVICE PERMANENT) 
 29 
 24 
 −5 
 −17.2 
 
 
 65 
 LIMITED EMERGENCY (SENIOR EXECUTIVE SERVICE NONPERMANENT) 
 0 
 2 
 2 
 +100.0 
 
 
 99 
 Unspecified 
 782 
 0 
 −782 
 −100.0 
 
 
 ** 
 Unspecified 
 77 
 0 
 −77 
 −100.0 
 
 
 REDACTED 
 REDACTED 
 0 
 771,384 
 771,384 
 +100.0 
 
 
 TOTAL 
 TOTAL 
 2,313,216 
 2,289,472 
 −23,744 
 −1.0

## Working Around Redaction: Age Groups

In [9]:
age_table_clean, excluded_agencies = create_age_groups_table_excluding_redacted_agencies(df_march, df_sept)
age_table_clean.show()

Federal Employment by Age Groups (Excluding Redacted Agencies) 
 
 
 September 2024 vs March 2025 Comparison 
 
 
 Category 
 Sep 2024 
 Mar 2025 
 Change 
 % Change 
 
 
 
 
 Less than 20 
 1,504 
 838 
 −666 
 −44.3 
 
 
 20-24 
 34,281 
 30,120 
 −4,161 
 −12.1 
 
 
 25-29 
 92,166 
 88,229 
 −3,937 
 −4.3 
 
 
 30-34 
 144,113 
 140,438 
 −3,675 
 −2.6 
 
 
 35-39 
 196,735 
 193,276 
 −3,459 
 −1.8 
 
 
 40-44 
 230,508 
 229,427 
 −1,081 
 −0.5 
 
 
 45-49 
 211,369 
 214,695 
 3,326 
 +1.6 
 
 
 50-54 
 206,922 
 204,025 
 −2,897 
 −1.4 
 
 
 55-59 
 190,929 
 189,532 
 −1,397 
 −0.7 
 
 
 60-64 
 139,672 
 137,711 
 −1,961 
 −1.4 
 
 
 65 or more 
 92,233 
 89,797 
 −2,436 
 −2.6 
 
 
 Unspecified 
 235 
 0 
 −235 
 −100.0 
 
 
 TOTAL 
 1,540,667 
 1,518,088 
 −22,579 
 −1.5 
 
 
 
 
 
 Excluded agencies: AF (AIR FORCE), AR (ARMY), DD (DEPARTMENT OF DEFENSE), NV (NAVY)

## Showing Salary Comparison: No Redaction Number Changes Besides Agency Exclusions

In [10]:
salary_table = create_salary_comparison_table(df_march, df_sept)
salary_table.show()

Salary Data Availability Comparison 
 
 
 September 2024 vs March 2025 (Excluding Redacted Agencies) 
 
 
 Category 
 Sep 2024 
 Mar 2025 
 Change 
 % Change 
 
 
 
 
 Total Records 
 1,540,667 
 1,518,088 
 −22,579 
 −1.5 
 
 
 Salary Available 
 1,275,482 
 1,236,580 
 −38,902 
 −3.0 
 
 
 Salary Redacted 
 0 
 281,390 
 281,390 
 +100.0 
 
 
 Salary Missing/Blank 
 265,185 
 118 
 −265,067 
 −100.0 
 
 
 
 
 
 Excluded agencies with redacted data: AF, AR, DD, NV

## Location Comparison: Some Additional Redactions, Mostly Shift From US-SUPPRESSED to Redacted

In [11]:
location_table = create_location_comparison_table(df_march, df_sept)
location_table.show()

Federal Employment by Location (All Locations, Excluding Redacted Agencies) 
 
 
 September 2024 vs March 2025 Comparison 
 
 
 Code 
 Category 
 Sep 2024 
 Mar 2025 
 Change 
 % Change 
 
 
 
 
 11 
 DISTRICT OF COLUMBIA 
 144,764 
 121,837 
 −22,927 
 −15.8 
 
 
 24 
 MARYLAND 
 98,966 
 97,129 
 −1,837 
 −1.9 
 
 
 06 
 CALIFORNIA 
 89,473 
 86,730 
 −2,743 
 −3.1 
 
 
 48 
 TEXAS 
 83,598 
 83,778 
 180 
 +0.2 
 
 
 12 
 FLORIDA 
 62,004 
 61,789 
 −215 
 −0.3 
 
 
 51 
 VIRGINIA 
 50,803 
 50,379 
 −424 
 −0.8 
 
 
 13 
 GEORGIA 
 47,995 
 48,135 
 140 
 +0.3 
 
 
 42 
 PENNSYLVANIA 
 42,314 
 42,098 
 −216 
 −0.5 
 
 
 36 
 NEW YORK 
 43,082 
 41,838 
 −1,244 
 −2.9 
 
 
 17 
 ILLINOIS 
 32,357 
 31,867 
 −490 
 −1.5 
 
 
 29 
 MISSOURI 
 29,950 
 29,753 
 −197 
 −0.7 
 
 
 37 
 NORTH CAROLINA 
 29,880 
 29,645 
 −235 
 −0.8 
 
 
 39 
 OHIO 
 29,192 
 28,669 
 −523 
 −1.8 
 
 
 08 
 COLORADO 
 29,481 
 28,453 
 −1,028 
 −3.5 
 
 
 53 
 WASHINGTON 
 27,338 
 26,176 
 −1,162 
 −4.3 
 
 
 04 
 ARIZONA 
 26,239 
 25,727 
 −512 
 −2.0 
 
 
 47 
 TENNESSEE 
 25,860 
 25,414 
 −446 
 −1.7 
 
 
 26 
 MICHIGAN 
 20,272 
 19,979 
 −293 
 −1.4 
 
 
 25 
 MASSACHUSETTS 
 19,327 
 18,926 
 −401 
 −2.1 
 
 
 49 
 UTAH 
 17,832 
 17,470 
 −362 
 −2.0 
 
 
 01 
 ALABAMA 
 17,296 
 17,125 
 −171 
 −1.0 
 
 
 41 
 OREGON 
 18,292 
 16,788 
 −1,504 
 −8.2 
 
 
 40 
 OKLAHOMA 
 16,770 
 16,688 
 −82 
 −0.5 
 
 
 35 
 NEW MEXICO 
 16,284 
 15,861 
 −423 
 −2.6 
 
 
 27 
 MINNESOTA 
 15,817 
 15,441 
 −376 
 −2.4 
 
 
 54 
 WEST VIRGINIA 
 15,479 
 15,272 
 −207 
 −1.3 
 
 
 55 
 WISCONSIN 
 15,571 
 15,137 
 −434 
 −2.8 
 
 
 45 
 SOUTH CAROLINA 
 14,217 
 14,222 
 5 
 +0.0 
 
 
 21 
 KENTUCKY 
 14,087 
 14,040 
 −47 
 −0.3 
 
 
 22 
 LOUISIANA 
 13,495 
 13,311 
 −184 
 −1.4 
 
 
 18 
 INDIANA 
 13,058 
 12,681 
 −377 
 −2.9 
 
 
 34 
 NEW JERSEY 
 12,494 
 12,249 
 −245 
 −2.0 
 
 
 20 
 KANSAS 
 11,381 
 11,051 
 −330 
 −2.9 
 
 
 32 
 NEVADA 
 11,228 
 10,966 
 −262 
 −2.3 
 
 
 05 
 ARKANSAS 
 10,959 
 10,752 
 −207 
 −1.9 
 
 
 28 
 MISSISSIPPI 
 10,358 
 10,238 
 −120 
 −1.2 
 
 
 30 
 MONTANA 
 10,043 
 8,806 
 −1,237 
 −12.3 
 
 
 16 
 IDAHO 
 9,450 
 8,605 
 −845 
 −8.9 
 
 
 19 
 IOWA 
 8,322 
 8,201 
 −121 
 −1.5 
 
 
 46 
 SOUTH DAKOTA 
 7,627 
 7,388 
 −239 
 −3.1 
 
 
 31 
 NEBRASKA 
 6,351 
 6,169 
 −182 
 −2.9 
 
 
 02 
 ALASKA 
 6,709 
 6,102 
 −607 
 −9.0 
 
 
 15 
 HAWAII 
 5,536 
 5,480 
 −56 
 −1.0 
 
 
 09 
 CONNECTICUT 
 5,137 
 5,434 
 297 
 +5.8 
 
 
 56 
 WYOMING 
 5,790 
 5,046 
 −744 
 −12.8 
 
 
 33 
 NEW HAMPSHIRE 
 4,187 
 4,108 
 −79 
 −1.9 
 
 
 38 
 NORTH DAKOTA 
 4,127 
 4,032 
 −95 
 −2.3 
 
 
 23 
 MAINE 
 4,093 
 3,948 
 −145 
 −3.5 
 
 
 44 
 RHODE ISLAND 
 3,377 
 3,340 
 −37 
 −1.1 
 
 
 50 
 VERMONT 
 2,917 
 2,866 
 −51 
 −1.7 
 
 
 10 
 DELAWARE 
 2,622 
 2,568 
 −54 
 −2.1 
 
 
 QA 
 QA-QATAR 
 5 
 0 
 −5 
 −100.0 
 
 
 MZ 
 MZ-MOZAMBIQUE 
 61 
 0 
 −61 
 −100.0 
 
 
 NH 
 NH-VANUATU 
 3 
 0 
 −3 
 −100.0 
 
 
 NI 
 NI-NIGERIA 
 39 
 0 
 −39 
 −100.0 
 
 
 NL 
 NL-NETHERLANDS 
 7 
 0 
 −7 
 −100.0 
 
 
 NO 
 NO-NORWAY 
 2 
 0 
 −2 
 −100.0 
 
 
 NP 
 NP-NEPAL 
 32 
 0 
 −32 
 −100.0 
 
 
 NS 
 NS-SURINAME 
 1 
 0 
 −1 
 −100.0 
 
 
 NU 
 NU-NICARAGUA 
 7 
 0 
 −7 
 −100.0 
 
 
 NZ 
 NZ-NEW ZEALAND 
 1 
 0 
 −1 
 −100.0 
 
 
 PA 
 PA-PARAGUAY 
 6 
 0 
 −6 
 −100.0 
 
 
 PE 
 PE-PERU 
 39 
 0 
 −39 
 −100.0 
 
 
 PK 
 PK-PAKISTAN 
 25 
 0 
 −25 
 −100.0 
 
 
 PL 
 PL-POLAND 
 17 
 0 
 −17 
 −100.0 
 
 
 PM 
 PM-PANAMA 
 24 
 0 
 −24 
 −100.0 
 
 
 PO 
 PO-PORTUGAL 
 1 
 0 
 −1 
 −100.0 
 
 
 PP 
 PP-PAPUA NEW GUINEA 
 2 
 0 
 −2 
 −100.0 
 
 
 PS 
 PS-PALAU 
 2 
 0 
 −2 
 −100.0 
 
 
 NG 
 NG-NIGER 
 10 
 0 
 −10 
 −100.0 
 
 
 ** 
 **-UNSPECIFIED 
 3,548 
 0 
 −3,548 
 −100.0 
 
 
 MY 
 MY-MALAYSIA 
 8 
 0 
 −8 
 −100.0 
 
 
 MD 
 MD-MOLDOVA 
 14 
 0 
 −14 
 −100.0 
 
 
 RM 
 RM-MARSHALL ISLANDS 
 2 
 0 
 −2 
 −100.0 
 
 
 LG 
 LG-LATVIA 
 3 
 0 
 −3 
 −100.0 
 
 
 LH 
 LH-LITHUANIA 
 1 
 0 
 −1 
